In [2]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pytesseract
from PIL import Image, ImageFilter
import cv2
from collections import Counter
import os
import time
import re
from datetime import datetime, timedelta
import pandas as pd
import csv
import ast
import pyodbc
import json
import concurrent.futures

In [6]:
def import_invoice_details_to_db():
    # Đọc file CSV
    df = pd.read_csv("all_invoice_info.csv", quotechar='"', sep=",", encoding="utf-8",dtype=str)
    def convert_to_datetime(date_str):
        try:
            if date_str and isinstance(date_str, str):  # Kiểm tra nếu date_str không rỗng và là chuỗi
                # Loại bỏ 3 ký tự cuối nếu chuỗi chứa "AM" hoặc "PM"
                cleaned_date_str = date_str[:-3].strip()
                # Chuyển đổi chuỗi thành datetime
                return datetime.strptime(cleaned_date_str, '%d/%m/%Y %H:%M:%S')  # Định dạng dd/mm/yyyy hh:mm:ss
            return None  # Trả về None nếu không có chuỗi hợp lệ
        except ValueError:
            return None  # Trả về None nếu không thể chuyển đổi
    df['created_at'] = df['created_at'].apply(convert_to_datetime)
    df['last_modified_at'] = df['last_modified_at'].apply(convert_to_datetime)

    columns_to_convert = ['co_info']
    for col in columns_to_convert:
        if col in df.columns:  # Kiểm tra nếu cột tồn tại
            df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)
    df = df.fillna('')


    # # Kết nối đến database master để tạo database mới
    # connection = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE=master;UID={dbusername};PWD={dbpassword}')
    # cursor = connection.cursor()

    # # Tạo database nếu chưa tồn tại
    # try:
    #     cursor.execute("IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'COData') "
    #                 "BEGIN "
    #                 "CREATE DATABASE COData; "
    #                 "END")
    #     connection.commit()  # Commit lệnh CREATE DATABASE
    # except Exception as e:
    #     print("Lỗi khi tạo cơ sở dữ liệu:", e)

    # Đóng kết nối và mở lại kết nối tới database mới
    # connection.close()

    # Kết nối đến database mới
    connection = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE=COData;UID={dbusername};PWD={dbpassword}')
    cursor = connection.cursor()

    # Tạo bảng invoice
    cursor.execute("""
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'invoice')
        BEGIN
            CREATE TABLE invoice (
                invoice_doc_id VARCHAR(50) PRIMARY KEY,
                status NVARCHAR(50),
                ordercode VARCHAR(20),
                invoice_code VARCHAR(50),
                companyname NVARCHAR(100),
                companytaxcode VARCHAR(50),
                companyaddress NTEXT,
                companyemail VARCHAR(100),
                amount VARCHAR(20),
                invoice_receip_no VARCHAR(50),
                invoice_address_label VARCHAR(50),
                another_email NVARCHAR(50),
                created_at DATETIME,
                last_modified_at DATETIME
            );
        END
    """)
    #tạo bảng invoice_co_info
    cursor.execute("""

        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'invoice_co_info')
        BEGIN
            CREATE TABLE invoice_co_info (
                invoice_doc_id VARCHAR(50),
                item_num smallint,
                co_num VARCHAR(50),
                fee_type NVARCHAR(50),
                fee NVARCHAR(50),
                CONSTRAINT FK_invoice_co_info FOREIGN KEY (invoice_doc_id) REFERENCES invoice(invoice_doc_id)
            );
        END                       
    """)

    connection.commit()  # Commit các lệnh tạo bảng

    # Đóng kết nối
    cursor.close()
    connection.close()

    print("Cơ sở dữ liệu và các bảng đã được tạo thành công.")

    # Kết nối đến SQL Server
    connection = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE=COData;UID={dbusername};PWD={dbpassword}')
    cursor = connection.cursor()

    # Chèn dữ liệu vào bảng invoice
    for idx,record in df.iterrows():
        # Chèn hoặc cập nhật dữ liệu vào bảng invoice
        cursor.execute("""
            MERGE INTO invoice AS target
            USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source
                (invoice_doc_id, status, ordercode, invoice_code, companyname, companytaxcode, companyaddress, companyemail, amount, invoice_receip_no, invoice_address_label, another_email, created_at, last_modified_at)
            ON target.invoice_doc_id = source.invoice_doc_id
            WHEN MATCHED THEN
                UPDATE SET
                    status = source.status,
                    ordercode = source.ordercode,
                    invoice_code = source.invoice_code,
                    companyname = source.companyname,
                    companytaxcode = source.companytaxcode,
                    companyaddress = source.companyaddress,
                    companyemail = source.companyemail,
                    amount = source.amount,
                    invoice_receip_no = source.invoice_receip_no,
                    invoice_address_label = source.invoice_address_label,
                    another_email = source.another_email,
                    created_at = source.created_at,
                    last_modified_at = source.last_modified_at
            WHEN NOT MATCHED THEN
                INSERT (invoice_doc_id, status, ordercode, invoice_code, companyname, companytaxcode, companyaddress, companyemail, amount, invoice_receip_no, invoice_address_label, another_email, created_at, last_modified_at)
                VALUES (source.invoice_doc_id, source.status, source.ordercode,source.invoice_code, source.companyname, source.companytaxcode, source.companyaddress, source.companyemail, source.amount, source.invoice_receip_no, source.invoice_address_label, source.another_email, source.created_at, source.last_modified_at);
        """, (
            record['doc_id'], record['status'], record['ordercode'], record['invoice_code'],
            record['companyname'], record['companytaxcode'], record['companyaddress'], 
            record['companyemail'], record['amount'], record['invoice_receip_no'], 
            record['invoice_address_label'], record['another_email'], 
            str(record['created_at']), str(record['last_modified_at'])
        ))


        # Chèn hoặc cập nhật dữ liệu vào bảng invoice_co_info
        for idx, goods in enumerate(record['co_info'][1:], 1):  # Bỏ qua dòng tiêu đề
            cursor.execute("""
                MERGE INTO invoice_co_info AS target
                USING (VALUES (?, ?, ?, ?, ?)) AS source
                    (invoice_doc_id, item_num, co_num, fee_type, fee)
                ON target.invoice_doc_id = source.invoice_doc_id AND target.co_num = source.co_num
                WHEN MATCHED THEN
                    UPDATE SET
                        item_num = source.item_num,
                        fee_type = source.fee_type,
                        fee = source.fee
                WHEN NOT MATCHED THEN
                    INSERT (invoice_doc_id, item_num, co_num, fee_type, fee)
                    VALUES (source.invoice_doc_id, source.item_num, source.co_num, source.fee_type, source.fee);
            """, (
                record['doc_id'], idx, goods[0], goods[2], goods[3]
            ))


    # Commit các thay đổi vào cơ sở dữ liệu
    connection.commit()

    # Đóng kết nối
    cursor.close()
    connection.close()

    print("Dữ liệu đã được chèn thành công.")


In [4]:
server = '3.35.252.62,1433'
dbusername = 'sa'
dbpassword = '1234QWER@'

In [7]:
import_invoice_details_to_db()

Cơ sở dữ liệu và các bảng đã được tạo thành công.
Dữ liệu đã được chèn thành công.


In [ ]:
def get_cookie(user_name, password):
    maxretries = 10
    retrytimes = 0
    while retrytimes < maxretries:  # Vòng lặp để thử đăng nhập liên tục
        retrytimes += 1
        # Cấu hình Selenium WebDriver với chế độ headless
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # Chế độ headless
        options.add_argument("--no-sandbox")  # Khắc phục lỗi sandbox trên Linux
        options.add_argument("--disable-dev-shm-usage")  # Tăng giới hạn bộ nhớ chia sẻ
        options.add_argument("--disable-gpu")  # Tắt GPU (không cần thiết trên server không có UI)
        options.add_argument("--window-size=1920x1080")  # Đặt kích thước màn hình để tránh lỗi định vị phần tử

        # Khởi tạo driver
        driver = webdriver.Chrome(options=options)

        try:
            # Mở trang đăng nhập
            url = "https://dichvucong.moit.gov.vn/Login.aspx"
            driver.get(url)

            # Chờ các phần tử cần thiết hiển thị
            wait = WebDriverWait(driver, 10)

            # Nhập username
            username_field = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_txtLoginName"]')))
            username_field.send_keys(user_name)

            # Nhập password
            password_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtPassword"]')
            password_field.send_keys(password)

            # Lấy URL của captcha từ Selenium
            captcha_image = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_imgCaptcha"]')))
            captcha_src = captcha_image.get_attribute("src")
            print(f"Captcha URL: {captcha_src}")

            # Tải ảnh captcha bằng session từ Selenium WebDriver
            captcha_path = download_captcha(captcha_src, driver)

            # Đọc mã captcha bằng Tesseract OCR
            captcha_code = run_multiple_captcha_attempts(captcha_path,attempts=10)
            print(f"Mã captcha OCR: {captcha_code}")

            # Nhập mã captcha
            captcha_field = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_txtCaptcha"]')
            captcha_field.send_keys(captcha_code)

            # Nhấn nút đăng nhập
            login_button = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_btnLogin"]')
            login_button.click()

            # Chờ tải xong sau khi đăng nhập
            time.sleep(5)

            # Kiểm tra thông báo lỗi captcha (nếu có)
            try:
                captcha_notice = driver.find_element(By.XPATH, '//*[@id="ctl00_cplhContainer_lblMsg"]').text
                if captcha_notice:
                    print(f"Thông báo: {captcha_notice}")
                    continue  # Quay lại đầu vòng lặp nếu đăng nhập thất bại
            except:
                pass

            # Kiểm tra đăng nhập thành công hay không
            if "Dashboard" in driver.page_source:
                print("Đăng nhập dichvucong.moit.gov.vn thành công!")

                # Đăng nhập sang ecosys
                next_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="ctl00_cplhContainer_grdViewDefault"]/tbody/tr[2]/td[4]/a')))
                next_button.click()

                # Lấy cookie
                cookie = f'{driver.get_cookies()[3]["name"]}={driver.get_cookies()[3]["value"]}'

                # Đóng driver và trả về cookie
                driver.quit()
                if os.path.exists("captcha_image.png"):
                    os.remove("captcha_image.png")
                print("Đăng nhập ecosys thành công!")    
                return cookie
            else:
                print("Đăng nhập thất bại. Thử lại...")

        except Exception as e:
            print(f"Lỗi xảy ra: {e}")

        finally:
            # Đóng trình duyệt nếu không thành công
            driver.quit()
            if os.path.exists("captcha_image.png"):
                os.remove("captcha_image.png")

In [2]:
maxretries = 10
retrytimes = 0
while retrytimes < maxretries:  # Vòng lặp để thử đăng nhập liên tục
    retrytimes += 1
    print(retrytimes)

1
2
3
4
5
6
7
8
9
10
